In [ ]:
import pandas as pd
import numpy as np

Load the Data set

In [ ]:
train  =  pd.read_csv("/content/Sales_forecasting_train.csv")
test   = pd.read_csv("/content/Sales_forecasting_test.csv") #Predcition dataset
EcoIndicators = pd.read_csv("/content/EconomicIndicators.csv")

In [ ]:
train.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry,Sales
0,0,CMP01,Q1,2.02,7.71,0.05,-0.04,CCC,Buy,South,Metal Fabrication,1517.0
1,1,CMP01,Q2,2.01,4.10,0.03,0.00,CCC,Hold,South,Metal Fabrication,2968.0
2,2,CMP01,Q3,2.02,6.79,0.06,-0.02,CCC,Buy,South,Metal Fabrication,1497.0
3,3,CMP01,Q4,1.98,3.97,0.01,0.02,CCC,Buy,South,Metal Fabrication,2929.0
4,4,CMP01,Q5,1.96,7.41,-0.07,0.02,CCC,Buy,South,Metal Fabrication,1452.0


In [ ]:
test.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry
0,7,CMP01,Q8,1.93,2.79,-0.03,-0.01,CCC,Buy,South,Metal Fabrication
1,8,CMP01,Q9,1.93,4.77,0.00,0.00,CCC,Buy,South,Metal Fabrication
2,16,CMP02,Q8,1.97,2.34,0.04,-0.03,A,Sell,West,Infrastructure
3,17,CMP02,Q9,1.93,4.14,0.04,0.01,BBB,Hold,West,Infrastructure
4,25,CMP03,Q8,0.67,NaN,-0.05,-0.01,BB,Buy,East,Infrastructure


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 675 non-null    int64  
 1   Company            675 non-null    object 
 2   Quarter            675 non-null    object 
 3   QuickRatio         675 non-null    float64
 4   InventoryRatio     523 non-null    float64
 5   RevenueGrowth      675 non-null    float64
 6   MarketshareChange  675 non-null    float64
 7   Bond rating        675 non-null    object 
 8   Stock rating       675 non-null    object 
 9   Region             675 non-null    object 
 10  Industry           675 non-null    object 
 11  Sales              525 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 63.4+ KB


In [ ]:
train.describe()

,ID,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Sales
count,675.000000,675.000000,523.000000,675.000000,675.000000,525.000000
mean,394.555556,1.603867,4.265124,-0.009733,-0.002904,3556.708571
std,204.960069,0.595615,3.108644,0.067390,0.017622,2028.059368
min,0.000000,0.500000,1.260000,-0.200000,-0.050000,864.000000
25%,216.500000,0.990000,2.630000,-0.070000,-0.015000,1992.000000
50%,433.000000,1.730000,3.420000,0.000000,0.000000,3007.000000
75%,579.000000,2.155000,4.725000,0.050000,0.010000,4523.000000
max,674.000000,2.490000,24.840000,0.080000,0.020000,11686.000000


In [ ]:
test.describe()

,ID,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange
count,150.000000,150.000000,118.000000,150.000000,150.000000
mean,340.500000,1.602867,3.962034,-0.001267,-0.002667
std,195.491765,0.585409,2.147586,0.059712,0.016330
min,7.000000,0.520000,1.390000,-0.180000,-0.050000
25%,172.000000,1.075000,2.610000,-0.040000,-0.010000
50%,340.500000,1.745000,3.505000,0.010000,0.000000
75%,509.000000,2.090000,4.575000,0.040000,0.010000
max,674.000000,2.480000,17.110000,0.080000,0.020000


In [ ]:
train.isnull().sum()

ID                     0
Company                0
Quarter                0
QuickRatio             0
InventoryRatio       152
RevenueGrowth          0
MarketshareChange      0
Bond rating            0
Stock rating           0
Region                 0
Industry               0
Sales                150
dtype: int64

In [ ]:
# Handle missing values in the 'Sales' column
train['Sales'].fillna(train['Sales'].median(), inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
# Create an instance of the KNNImputer
imputer = KNNImputer()

# Fit and transform the 'InventoryRatio' column in the training data
train['InventoryRatio'] = imputer.fit_transform(train['InventoryRatio'].values.reshape(-1, 1))

# Transform the 'InventoryRatio' column in the test data
test['InventoryRatio'] = imputer.transform(test['InventoryRatio'].values.reshape(-1, 1))

In [ ]:
# Define mapping dictionary for months to quarter numbers
month_to_quarter = {
    1: 1, 2: 1, 3: 1,
    4: 2, 5: 2, 6: 2,
    7: 3, 8: 3, 9: 3,
    10: 4, 11: 4, 12: 4,
    13: 5, 14: 5, 15: 5,
    16: 6, 17: 6, 18: 6,
    19: 7, 20: 7, 21: 7,
    22: 8, 23: 8, 24: 8,
    25: 9, 26: 9, 27: 9,
    28: 10, 29: 10, 30: 10,
}


In [ ]:
# Feature Engineering
train['Quarter_Num'] = train['Quarter'].str[1].astype(int)
test['Quarter_Num'] = test['Quarter'].str[1].astype(int)

In [ ]:
# Map Month column to Quarter_Num based on the mapping dictionary
EcoIndicators['Quarter_Num'] = EcoIndicators['Month'].map(month_to_quarter)

In [ ]:
train.dropna(subset=['Sales'], inplace=True)

In [ ]:
# Merge train_df and economic_indicators_df based on the Quarter_Num column
train_merged = pd.merge(train, EcoIndicators, how='left', on='Quarter_Num')

# Merge test_df and economic_indicators_df based on the Quarter_Num column
test_merged = pd.merge(test, EcoIndicators, how='left', on='Quarter_Num')


In [ ]:
train_merged.drop('Month', axis=1, inplace=True)
test_merged.drop('Month', axis=1, inplace=True)

In [ ]:
train.isnull().sum()

ID                   0
Company              0
Quarter              0
QuickRatio           0
InventoryRatio       0
RevenueGrowth        0
MarketshareChange    0
Bond rating          0
Stock rating         0
Region               0
Industry             0
Sales                0
Quarter_Num          0
dtype: int64

In [ ]:
test.isnull().sum()

ID                   0
Company              0
Quarter              0
QuickRatio           0
InventoryRatio       0
RevenueGrowth        0
MarketshareChange    0
Bond rating          0
Stock rating         0
Region               0
Industry             0
Quarter_Num          0
dtype: int64

In [ ]:
train_data = train_merged
test_data = test_merged

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025 entries, 0 to 2024
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  2025 non-null   int64  
 1   Company             2025 non-null   object 
 2   Quarter             2025 non-null   object 
 3   QuickRatio          2025 non-null   float64
 4   InventoryRatio      2025 non-null   float64
 5   RevenueGrowth       2025 non-null   float64
 6   MarketshareChange   2025 non-null   float64
 7   Bond rating         2025 non-null   object 
 8   Stock rating        2025 non-null   object 
 9   Region              2025 non-null   object 
 10  Industry            2025 non-null   object 
 11  Sales               2025 non-null   float64
 12  Quarter_Num         2025 non-null   int64  
 13  Consumer Sentiment  2025 non-null   float64
 14  Interest Rate       2025 non-null   float64
 15  PMI                 2025 non-null   float64
 16  Money 

In [ ]:
# Drop duplicate 'ID' values from the test dataset
test_data = test_data.drop_duplicates(subset='ID', keep='first')

In [ ]:
company_dummies = pd.get_dummies(train_data['Company'], prefix='Company')
train_data = pd.concat([train_data, company_dummies], axis=1)
train_data.drop('Company', axis=1, inplace=True)

In [ ]:
bond_rating_mapping = {'AAA': 7, 'AA': 6, 'A': 5, 'BBB': 4, 'BB': 3, 'B': 2, 'CCC': 1} #checked
train_data['Bond rating'] = train_data['Bond rating'].map(bond_rating_mapping)

In [ ]:
stock_rating_mapping = {'Strong Buy': 5, 'Buy': 4, 'Hold': 3, 'Sell': 2, 'Strong Sell': 1} #Checked
train_data['Stock rating'] = train_data['Stock rating'].map(stock_rating_mapping)

In [ ]:
region_dummies = pd.get_dummies(train_data['Region'], prefix='Region') #Checked
train_data = pd.concat([train_data, region_dummies], axis=1)
train_data.drop('Region', axis=1, inplace=True)

In [ ]:
industry_dummies = pd.get_dummies(train_data['Industry'], prefix='Industry')
train_data = pd.concat([train_data, industry_dummies], axis=1)
train_data.drop('Industry', axis=1, inplace=True)

In [ ]:
# Company
company_dummies_test = pd.get_dummies(test_data['Company'], prefix='Company')
test_data = pd.concat([test_data, company_dummies_test], axis=1)
test_data.drop('Company', axis=1, inplace=True)

# Bond Rating
test_data['Bond rating'] = test_data['Bond rating'].map(bond_rating_mapping)

# Stock Rating
test_data['Stock rating'] = test_data['Stock rating'].map(stock_rating_mapping)

# Region
region_dummies_test = pd.get_dummies(test_data['Region'], prefix='Region')
test_data = pd.concat([test_data, region_dummies_test], axis=1)
test_data.drop('Region', axis=1, inplace=True)

# Industry
industry_dummies_test = pd.get_dummies(test_data['Industry'], prefix='Industry')
test_data = pd.concat([test_data, industry_dummies_test], axis=1)
test_data.drop('Industry', axis=1, inplace=True)

In [ ]:
# Replace spaces with underscores in column names
train_data.columns = train_data.columns.str.replace(' ', '_')
test_data.columns = test_data.columns.str.replace(' ', '_')

In [ ]:
# Drop the 'ID' column from the train dataset
#train_data.drop('ID', axis=1, inplace=True)
train_data.drop('Quarter', axis=1, inplace=True)
test_data.drop('Quarter', axis=1, inplace=True)


In [ ]:
# Selecting only numerical columns
numerical_columns = train_data.select_dtypes(include=['float64', 'int64', 'int32','bool']).columns

# Calculate correlation coefficients
correlation_matrix = train_data[numerical_columns].corr()

# Extract correlation coefficients of numerical variables with 'Sales'
sales_correlation = correlation_matrix['Sales']

# Display correlation coefficients sorted in descending order
sales_correlation = sales_correlation.abs().sort_values(ascending=False)

print("Correlation coefficients with 'Sales':")
print(sales_correlation)

Correlation coefficients with 'Sales':
Sales                  1.000000
Company_CMP27          0.227660
Company_CMP05          0.223082
Company_CMP30          0.218097
Company_CMP18          0.197307
                         ...   
Company_CMP65          0.005429
Stock_rating           0.004940
Quarter_Num            0.002569
Company_CMP69          0.002336
Industry_Automobile    0.001814
Name: Sales, Length: 100, dtype: float64


In [ ]:
# Select columns with positive correlation
positive_correlation_columns = sales_correlation[sales_correlation > 0].index

# Create a new dataframe with only the columns with positive correlation
train_data_pcr = train_data[positive_correlation_columns]

# Display the new dataframe
print(train_data_pcr.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025 entries, 0 to 2024
Data columns (total 100 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Sales                       2025 non-null   float64
 1   Company_CMP27               2025 non-null   bool   
 2   Company_CMP05               2025 non-null   bool   
 3   Company_CMP30               2025 non-null   bool   
 4   Company_CMP18               2025 non-null   bool   
 5   Company_CMP29               2025 non-null   bool   
 6   Company_CMP28               2025 non-null   bool   
 7   Company_CMP20               2025 non-null   bool   
 8   Region_West                 2025 non-null   bool   
 9   Region_North                2025 non-null   bool   
 10  Company_CMP33               2025 non-null   bool   
 11  InventoryRatio              2025 non-null   float64
 12  QuickRatio                  2025 non-null   float64
 13  Company_CMP15               2025

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Assuming X and y are your feature matrix and target variable
# Splitting the data into train and test sets
X_train = train_data_pcr.drop('Sales', axis=1)
y_train = train_data_pcr['Sales']

# Initialize the linear regression model
model = LinearRegression()

# Initialize Sequential Feature Selector
sfs = SFS(model,
          k_features='best',  # Find the best set of features
          forward=True,  # Forward selection
          floating=False,
          scoring='neg_mean_squared_error',  # Use mean squared error as the evaluation metric
          cv=5)

# Fit the Sequential Feature Selector
sfs = sfs.fit(X_train, y_train)

# Get the selected features
selected_features = list(sfs.k_feature_names_)



In [134]:
# Get the selected features
selected_features = list(sfs.k_feature_names_)

In [135]:
print(selected_features)

['Company_CMP27', 'Company_CMP05', 'Company_CMP29', 'Company_CMP20', 'Region_West', 'Region_North', 'Company_CMP33', 'InventoryRatio', 'QuickRatio', 'Company_CMP15', 'Company_CMP14', 'Company_CMP53', 'Company_CMP50', 'Company_CMP11', 'RevenueGrowth', 'Company_CMP03', 'Company_CMP47', 'Company_CMP08', 'Company_CMP52', 'Company_CMP34', 'Industry_Metal_Fabrication', 'Company_CMP63', 'MarketshareChange', 'Company_CMP01', 'Company_CMP57', 'Company_CMP46', 'Company_CMP43', 'Company_CMP12', 'Company_CMP75', 'Company_CMP09', 'Company_CMP55', 'Company_CMP51', 'Money_Supply', 'Company_CMP62', 'Company_CMP59', 'WestEAI', 'EastEAI', 'Company_CMP39', 'SouthEAI', 'NationalEAI', 'Consumer_Sentiment', 'Company_CMP04', 'Company_CMP17', 'NorthEAI', 'Company_CMP74', 'Company_CMP13', 'Company_CMP37', 'Company_CMP10', 'Company_CMP45', 'Company_CMP58', 'Company_CMP35', 'Company_CMP25', 'PMI', 'Company_CMP56']


In [152]:
new_Test = test_data

In [146]:
from sklearn.preprocessing import StandardScaler

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_selected_scaled = scaler.fit_transform(X_train[selected_features])

# Transform the test data using the fitted scaler
X_test_scaled = scaler.transform(new_Test[selected_features])


In [147]:
from sklearn.ensemble import RandomForestRegressor
# Create and train the Random Forest model using selected features
rf_model = RandomForestRegressor(n_estimators=8000, random_state=60)
rf_model.fit(X_train_selected_scaled, y_train)


RandomForestRegressor(n_estimators=8000, random_state=60)

In [148]:
from sklearn.metrics import mean_absolute_error

# Make predictions on the training set
predictions_train = rf_model.predict(X_train_selected_scaled)

# Calculate Mean Absolute Error (MAE) on the training set
mae_train = mean_absolute_error(y_train, predictions_train)

# Print the MAE
print("Training Mean Absolute Error (MAE):", mae_train)


Training Mean Absolute Error (MAE): 355.8304058680099


In [153]:
# Evaluate the model
y_pred = rf_model.predict(new_Test[selected_features])


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [154]:
# Create a DataFrame with the test predictions and 'ID' column
submission_df = pd.DataFrame({'ID': test_data['ID'], 'Sales': y_pred})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('Submission24.csv', index=False)

In [151]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Assuming X_train_selected_scaled and y_train are your training features and target variable respectively

# Create and train the Random Forest model using selected features
rf_model = RandomForestRegressor(n_estimators=8000, random_state=60)
rf_model.fit(X_train_selected_scaled, y_train)

# Make predictions on the training set
predictions_train = rf_model.predict(X_train_selected_scaled)

# Calculate Mean Absolute Error (MAE) on the training set
mae_train = mean_absolute_error(y_train, predictions_train)

# Print the MAE
print("Training Mean Absolute Error (MAE):", mae_train)


Training Mean Absolute Error (MAE): 355.8304058680099


In [ ]:
# Predict on test set
y_pred = model.predict(new_Test[selected_features])


In [155]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Initialize XGBoost model
model = xgb.XGBRegressor(n_estimators=8000, learning_rate=0.5, max_depth=4, random_state=60)

# Fit the model
model.fit(X_train_selected_scaled, y_train)

# Predict on the training set
y_pred_train = model.predict(X_train_selected_scaled)

# Calculate Mean Absolute Error (MAE) on the training set
mae_train = mean_absolute_error(y_train, y_pred_train)

# Print the MAE
print("Mean Absolute Error (MAE) on training set:", mae_train)


Mean Absolute Error (MAE) on training set: 278.12630820191936


In [ ]:
X_train_selected = X_train[selected_features]

In [156]:
# Predict on test set
y_pred_xgboost = model.predict(new_Test[selected_features])

In [157]:
# Create a DataFrame with the test predictions and 'ID' column
submission_df = pd.DataFrame({'ID': test_data['ID'], 'Sales': y_pred_xgboost})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('Submission25.csv', index=False)